In [ ]:
import jax
import jax.numpy as jnp

def fgx():
    print("*" * 50)  # 使用横线
    print("我是分割线".center(47, "="))  # 居中显示并填充等号
    print("*" * 50)  # 使用星号

In [ ]:
# jit 中的打印语句

# 不要使用普通的 print 语句，因为 jit 编译后，print 语句不会被执行
# 而且输出的并不是想要的值
@jax.jit
def f_wrong(x):
    print("print(x) ->", x)
    y = jnp.sin(x)
    print("print(y) ->", y)
    return y

f_wrong(2.)
f_wrong(3.) #注意：这里的print不会被执行，因为 print 只在第一次编译时执行


@jax.jit
def f_correct(x):
    jax.debug.print("jax.debug.print(x) -> {x}", x=x)
    y = jnp.sin(x)
    jax.debug.print("jax.debug.print(y) -> {y}", y=y)
    return y

f_correct(2.)
f_correct(3.)

In [ ]:
# vmap 中的打印语句

xs = jnp.array([0., 1., 1.5])
xs_more = jnp.arange(3., 6.)

def f_correct(x):
    jax.debug.print("jax.debug.print(x) -> {}", x)
    y = jnp.sin(x)
    jax.debug.print("jax.debug.print(y) -> {}", y)
    return y

result = jax.vmap(f_correct)(xs)

fgx()

def f_wrong(x):
    print("jax.debug.print(x) ->", x)
    y = jnp.sin(x)
    print("jax.debug.print(y) ->", y)
    return y

jf = jax.vmap(f_wrong)
result = jf(xs)
result = jf(xs_more) #与 jit 不同，这里第二次调用也会执行， 但是使用普通的打印并不会出现我们

In [ ]:
# lax.map 中的打印语句
# 这里的 lax.map 类似于 vmap
# 但是 lax.map 是顺序执行的，而 vmap 是并行执行的
# 注意到在下面的输出中，先打印了 y 的值，然后才打印 x 的值，这是因为打印是按照实际执行的顺序进行的，而不是代码顺序，好像是计算图就是这么构造的

result = jax.lax.map(f_correct, xs)
print(result)

In [ ]:
# 据说 vmap 是并行执行的，所以打印顺序可能会乱掉
# 但是实际上，我在 CPU 上进行了尝试，依然是顺序执行的
# 在 GPU 上可能会并行执行

import jax
import jax.numpy as jnp
import time
import random

# 定义一个函数，带有调试打印和随机延迟
def f(x):
    time.sleep(random.uniform(0, 1))  # 随机延迟
    jax.debug.print("Processing x: {}", x)
    return x ** 2

# 输入数组
xs = jnp.array([3, 1, 4, 1, 5, 9, 2, 6, 5, 3, 5])

# 使用 vmap 并行执行
result = jax.vmap(f)(xs)

print("Result:", result)

In [ ]:
# 注意实际的打印顺序
# 只有前向传播有打印

def f(x):
    jax.debug.print("jax.debug.print(x) -> {}", x)
    return x ** 2

# 一阶导数
df = jax.grad(f)
print("我是分割线")
print("First derivative at x=2: {}", df(3.))

# 二阶导数
d2f = jax.grad(df)
jax.debug.print("我是分割线")
jax.debug.print("Second derivative at x=2: {}", d2f(3.))

In [ ]:
# 下面的代码中，如果不加 ordered=True，打印的顺序可能会乱掉

@jax.jit
def f(x, y):
  jax.debug.print("jax.debug.print(x) -> {}", x, ordered=True)
  jax.debug.print("jax.debug.print(y) -> {}", y, ordered=True)
  return x + y

f(1, 2)

In [ ]:
# 所谓的 pdb-like debugging，所谓的pdb就是 python debug
# 注意，在输入框中可以尝试 y*z，按 c 退出
# 我尝试过了，IDE 中的断点调试器是无法使用的，并不能输出变量的值

@jax.jit
def f(x):
  y, z = jnp.sin(x), jnp.cos(x)
  jax.debug.breakpoint()
  return y * z
f(2.) # ==> Pauses during execution

In [ ]:
# 使用运行时判断语句加断点
# 注意，下面代码中的两个小函数，在原来的教程中是放在函数里面的

def true_fn(x):
    pass

def false_fn(x):
    jax.debug.breakpoint()

def breakpoint_if_nonfinite(x):
    is_finite = jnp.isfinite(x).all()
    jax.lax.cond(is_finite, true_fn, false_fn, x)

@jax.jit
def f(x, y):
  z = x / y
  breakpoint_if_nonfinite(z)
  return z

f(2., 1.) # ==> No breakpoint

In [ ]:
# 使用 jax.debug.callback 的例子
# callback 是回调函数的意思，也就是说在函数执行到某个地方时，调用一个函数进行排错debug

def log_value(x):
  print(f'Logged value: {x}')
  # logging.warning(f'Logged value: {x}') # 教程原来提供的是这句，其优先级比较高，所以顺序会乱掉
  # 需要导入 import logging

@jax.jit
def f(x):
  jax.debug.callback(log_value, x)
  return x

jax.debug.print("我是分割线1", ordered=True)
f(1.0) # ==> Logs value
jax.debug.print("我是分割线2", ordered=True)
x = jnp.arange(5.0)
jax.vmap(f)(x);
print("我是分割线3")
jax.grad(f)(1.0);